## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 3.4 MB 8.5 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 1.2 MB 8.8 MB/s 


## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

## **Setting up Stemmer**

In [5]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
               

## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [8]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam93,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [9]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataframe.shape

(5880, 3)

In [11]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(5877, 3)

In [13]:
text = dataframe['text']

In [14]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [15]:
label = dataframe['category']

In [16]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [17]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [18]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [19]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [20]:
! chmod +x tamil-stemmer-build/snowball-with-tamil/stemwords

In [21]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [22]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [23]:
with open('./MuRIL/tamil_stopwords.txt', encoding = 'utf-8') as f:
    tamil_stopwords = f.readlines()
    for i in range(len(tamil_stopwords)):
        tamil_stopwords[i] = re.sub('\n','',tamil_stopwords[i])
stopwords = tamil_stopwords

# Function for removing stop words
def stopwords_remove(text):
    # Appending words which are not stop words
    text = text.split(" ")
    removed = [s for s in text if s not in stopwords]  
    return removed

In [24]:
text = [stopwords_remove(s) for s in text]
for i in range(len(text)):
  text[i] = (" ".join(text[i]))

In [25]:
len(text)

5877

In [ ]:
# text = np.array(text)
# text

## **Feature Extraction**

In [26]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [28]:
print(tokenized_input)

{'input_ids': tensor([[  104, 78708, 40896, 56772, 26847,  7772,  6396, 14801, 65001,   105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [29]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0019,  0.0018,  0.0002,  ..., -0.0100, -0.0018, -0.0032],
         [ 0.0044, -0.0005,  0.0013,  ..., -0.0104,  0.0030, -0.0012],
         [ 0.0072,  0.0032,  0.0010,  ..., -0.0176, -0.0016,  0.0021],
         ...,
         [-0.0112, -0.0004, -0.0028,  ..., -0.0070, -0.0009, -0.0035],
         [-0.0089,  0.0077, -0.0021,  ..., -0.0112,  0.0031, -0.0095],
         [-0.0019,  0.0018,  0.0002,  ..., -0.0100, -0.0018, -0.0032]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.4604e-03,  1.9281e-02, -7.5585e-03,  1.5480e-02, -7.2021e-03,
         -7.9304e-03, -1.7455e-03, -3.7400e-03, -3.2170e-03, -9.1676e-03,
         -9.6881e-03,  5.1594e-03,  1.5177e-03,  9.4433e-05, -4.6815e-03,
          1.7146e-02,  2.0318e-03, -8.6893e-03,  1.4831e-04, -6.0019e-03,
          6.1142e-03,  5.5887e-04,  6.6706e-03, -2.4938e-02,  5.0531e-03,
          1.4476e-03, -7.6066e-03, -6.1155e-03,  8.6085e-04, -2.312

In [30]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [31]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [32]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output1.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output2.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output3.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output4.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output5.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output6.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output7.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output8.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output9.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output10.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output11.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output12.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output13.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output14.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output15.p

In [33]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output1.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output2.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output3.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output4.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output5.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output6.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output7.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output8.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output9.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output10.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output11.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output12.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output13.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output14.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TrainData/output15.p

In [34]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [35]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [36]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
X_train.shape

(4701, 768)

In [39]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [40]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [41]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [42]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.92      0.75      0.83       950
           1       0.41      0.72      0.52       226

    accuracy                           0.74      1176
   macro avg       0.66      0.74      0.67      1176
weighted avg       0.82      0.74      0.77      1176



## **Stochastic Gradient Descent**

In [43]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **K Nearest Neighbours**

In [44]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.85      0.95      0.89       950
           1       0.55      0.27      0.36       226

    accuracy                           0.82      1176
   macro avg       0.70      0.61      0.63      1176
weighted avg       0.79      0.82      0.79      1176



## **Decision tree**

In [45]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85       950
           1       0.37      0.41      0.39       226

    accuracy                           0.75      1176
   macro avg       0.61      0.62      0.62      1176
weighted avg       0.76      0.75      0.76      1176



## **Random Forest**

In [47]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       950
           1       0.66      0.15      0.25       226

    accuracy                           0.82      1176
   macro avg       0.75      0.57      0.58      1176
weighted avg       0.80      0.82      0.77      1176



## **Majority Voting**

In [48]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only DT, KNN & NB in predictions due to poor performance
    predictions = [dt_pred[i], knn_pred[i], nb_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       950
           1       0.49      0.48      0.48       226

    accuracy                           0.80      1176
   macro avg       0.68      0.68      0.68      1176
weighted avg       0.80      0.80      0.80      1176



## **Loading Testing Data**

In [49]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [50]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_579,ஓத்தா தல வெறியன் டா #Mohan நீ தான் டா நீ மட்டு...
freq,1,1


In [51]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [52]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [53]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

In [54]:
! chmod +x tamil-stemmer-build/snowball-with-tamil/stemwords

In [55]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [56]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

In [57]:
text_test = [stopwords_remove(s) for s in text_test]
for i in range(len(text_test)):
  text_test[i] = (" ".join(text_test[i]))

## **Test Feature Extraction**

In [58]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [60]:
print(tokenized_input)

{'input_ids': tensor([[   104,  20005,   7848,  26657,   1394,   1895,   4365,    806,   1844,
          40017,  14088,  25557,   2131,  20005,   7848,  34654,   9278,  92683,
           2921, 167797,    105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [61]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0016,  0.0020,  0.0003,  ..., -0.0121, -0.0020, -0.0029],
         [ 0.0013,  0.0058, -0.0019,  ..., -0.0112, -0.0003, -0.0005],
         [ 0.0109,  0.0046,  0.0033,  ..., -0.0126, -0.0018, -0.0025],
         ...,
         [-0.0025, -0.0027, -0.0009,  ..., -0.0112, -0.0001, -0.0022],
         [-0.0035,  0.0009, -0.0021,  ..., -0.0192, -0.0007, -0.0020],
         [-0.0016,  0.0020,  0.0003,  ..., -0.0121, -0.0020, -0.0029]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.3762e-03,  1.8877e-02, -7.7534e-03,  1.6113e-02, -7.8707e-03,
         -8.0824e-03, -1.5879e-03, -4.1406e-03, -3.1543e-03, -9.0057e-03,
         -1.0355e-02,  5.9202e-03,  1.5974e-03,  6.7547e-04, -4.3367e-03,
          1.7603e-02,  2.4834e-03, -9.2536e-03,  1.4160e-04, -5.6650e-03,
          6.4071e-03,  9.5262e-04,  7.2215e-03, -2.5743e-02,  5.1022e-03,
          7.6533e-04, -6.7919e-03, -6.2313e-03,  2.0061e-03, -2.312

In [62]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [63]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [64]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output1.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output2.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output3.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output4.pickle done


In [65]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output1.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output2.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output3.pickle done
./MuRIL/MuRIL-Stopword-Stem-Embedded-TestData/output4.pickle done


In [66]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [67]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [68]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [69]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [nb_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [70]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [71]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [72]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.94      0.76      0.84       536
           1       0.41      0.76      0.53       118

    accuracy                           0.76       654
   macro avg       0.67      0.76      0.68       654
weighted avg       0.84      0.76      0.78       654



In [75]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       536
           1       0.61      0.28      0.38       118

    accuracy                           0.84       654
   macro avg       0.73      0.62      0.65       654
weighted avg       0.81      0.84      0.81       654



In [77]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.86      0.85      0.85       536
           1       0.34      0.35      0.34       118

    accuracy                           0.76       654
   macro avg       0.60      0.60      0.60       654
weighted avg       0.76      0.76      0.76       654



In [78]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       536
           1       0.67      0.20      0.31       118

    accuracy                           0.84       654
   macro avg       0.76      0.59      0.61       654
weighted avg       0.82      0.84      0.80       654



In [79]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       536
           1       0.46      0.45      0.45       118

    accuracy                           0.80       654
   macro avg       0.67      0.67      0.67       654
weighted avg       0.80      0.80      0.80       654

